In [29]:
# Ensure the project root is in the Python path
import sys
sys.path.append('/Users/tl759k/Documents/GitHub/work/cursor-analytics')

# Activate the virtual environment (if not already activated)
# This is typically done outside the notebook, but we ensure the path is correct here
import os
os.system('source /Users/tl759k/Documents/GitHub/work/cursor-analytics/venv/bin/activate')

# Re-import the SnowflakeHook after ensuring the path is set
from utils.snowflake_connection import SnowflakeHook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import plotly.graph_objects as go
import seaborn as sns
import statsmodels.formula.api as smf
from decimal import Decimal
import plotly.express as px


# Create the Snowflake connection hook once
snowhook = SnowflakeHook()

# Function for queries
def load_query(file_path):
    with open(file_path, 'r') as file:
        query = file.read()
    df = snowhook.query_snowflake(query, method='pandas')

    # format Decimal -> float
    for col in df.columns:
        if df[col].dtype == 'object' and isinstance(df[col].dropna().iloc[0], Decimal):
            df[col] = df[col].astype(float)
    return df


2025-09-12 10:52:03,127 - utils.snowflake_connection - ERROR - Failed to create optimized Spark session: Java gateway process exited before sending its port number
2025-09-12 10:52:03,128 - utils.snowflake_connection - ERROR - Failed to create Spark session: Java gateway process exited before sending its port number


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/tl759k/Documents/GitHub/work/cursor-analytics/venv/lib/python3.11/site-packages/pyspark/bin/spark-class: line 96: CMD: bad array subscript
head: illegal line count -- -1


### Load data

In [30]:
df_spend = load_query('historical_quarterly_spend.sql')
df_apps = load_query('historical_quarterly_apps.sql')
# df_new_dx = load_query('historical_quarterly_new_dx.sql')
df_wad = load_query('historical_quarterly_wad.sql')

2025-09-12 10:52:03,670 - utils.snowflake_connection - INFO - Successfully connected to Snowflake
2025-09-12 10:52:03,671 - utils.snowflake_connection - INFO - Executing query (pandas)
2025-09-12 10:52:03,862 - utils.snowflake_connection - INFO - Executing query (pandas)
2025-09-12 10:52:04,010 - utils.snowflake_connection - INFO - Executing query (pandas)


In [31]:
df_combined = df_spend.merge(df_apps, on=['quarter', 'submarket_id'], how='left')
df_combined = df_combined.merge(df_wad, on=['quarter', 'submarket_id'], how='left')
df_combined.head()

,quarter,submarket_id,avg_weekly_spend,avg_weekly_paid_media_apps,online_hours_avg,dx_cnt_avg,hours_per_dx
0,2024-01-01,0,"4,980,801","51,563","475,507","41,050",12
1,2024-01-01,5,"101,213",777,"7,744",639,12
2,2024-01-01,7,"28,090",368,"4,119",367,11
3,2024-01-01,81,"56,844",263,"2,781",259,11
4,2024-04-01,0,"2,393,344","31,258","277,860","22,838",12


In [32]:
df_combined_sm0 = df_combined[df_combined['submarket_id'] == 0]

In [33]:

df_combined_sm0['cpa'] = df_combined_sm0['avg_weekly_spend'] / df_combined_sm0['avg_weekly_paid_media_apps']
pd.options.display.float_format = '{:,.0f}'.format
df_combined_sm0

/var/folders/d8/0ml7793s3d1c9g2vzlb7wkb40000gn/T/ipykernel_42342/3241890153.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_sm0['cpa'] = df_combined_sm0['avg_weekly_spend'] / df_combined_sm0['avg_weekly_paid_media_apps']


,quarter,submarket_id,avg_weekly_spend,avg_weekly_paid_media_apps,online_hours_avg,dx_cnt_avg,hours_per_dx,cpa
0,2024-01-01,0,"4,980,801","51,563","475,507","41,050",12,97
4,2024-04-01,0,"2,393,344","31,258","277,860","22,838",12,77
8,2024-07-01,0,"1,712,489","29,832","193,919","16,415",12,57
12,2024-10-01,0,"4,638,879","43,853","293,708","27,754",11,106
16,2025-01-01,0,"6,058,800","50,527","368,043","34,089",11,120
20,2025-04-01,0,"3,460,665","40,403","273,890","27,540",10,86
